# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
cities_df = pd.read_csv('../output_data/cities.csv', index_col = False)
cities_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=g_key)

In [19]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity_weight = cities_df["Humidity"].astype(float)

In [20]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
# A max temperature lower than 80 degrees but higher than 74.
# Wind speed less than 10 mph.
# Zero cloudiness.
address =  ['ponta do sol', 'cidreira', 'morondava']
hotel_df = cities_df[(cities_df['Max Temp'] > 70) & (cities_df['Max Temp'] < 80)& (cities_df['Wind Speed'] < 10)& (cities_df['Cloudiness'] < 1)]

hotel_clean_df = hotel_df.dropna()
hotel_clean_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
# geocoordinates
target_coordinates = ['-20.63, 	-46.00','-30.17, -50.22','-20.30, 44.28']
# target_search = "BR"
target_radius = 5000
target_type = "hotels"

# set up a parameters dictionary
params = {
    "location": target_coordinates[1],
#     "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [36]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwI4Ass0ytxUN2TJ9ph_vHE5RMq_wXAjlrWSYPUKlb_-2zXrBdkJAUsjr4brNKV3aaqk4IrZH-Fzs0OJ39vCNJGGApLM37sFBIokK2IYTL9xGnpStgdqngIyz2YUcrHaWPANuLpWRdQqzOqnqBWDBP3GSNhdz9JSmyvfBobPmZBcWKdACVcNMwGnooHT3ROGnVuO1IEDi9tyvnla5tRd6MZPdhyWvV9epl5zVjHTrSGT9XX5ddUTV2tszslTWWksfpMiVnZ-9MBhUzfu2kAzYzYTg0CJDbjRTE85uarsYgWEC4CzZSdzy4QTGpYDhnzGxVVk8arDIahVCimr_ytjKMNKvOSmokH-dIUtebmiZ8Onk9eMaIV3GRMsUqoA0p7Q8AGfo7RGYItwUxzM-MNP7oLLpm4sclhVY7a_vBRGmYqwCtH0QnluewQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -30.1162926,
                    "lng": -50.2862819
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.043783692517,
                        "lng": -50.17074083566425
                    },
                    "southwest": {
                        "lat": -30.23734717244479,
                        "lng": -50.3942816687

In [44]:
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

CIDREIRA
CIDREIRA


In [55]:
try:
    narrowed_city_df = hotel_df.assign(**{'Hotel Name' : address })
 
    narrowed_city_df
except (KeyError, IndexError, ValueError):
    print("Missing field/result...skipping.")

Missing field/result...skipping.


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [57]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer([(-34.42, 19.24),(32.68, -115.50),(19.71,-155.08)])
# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))